In [8]:
#Import Dependencies
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd 
from config import password


#Create connection to AWS database
ENDPOINT="fantasyfootball.ckswyhqaro8q.us-east-2.rds.amazonaws.com"
PORT="5432"
USR="DreamTeam"
PSSWD=password
DBNAME="postgres"



In [9]:
# Edit display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
#create connection and open engine 
#con = "host='localhost' dbname='my_database' user='postgres' password='secret'
con= "postgresql://{}:{}@{}:{}/{}".format(USR, PSSWD, ENDPOINT, PORT, DBNAME)
db = create_engine(con)


In [11]:
# Query db w/ joins to get full stat tables
positions = ['qb', 'rb', 'te', 'wr']
df_list = []
for x in positions:
    df = f'{x}_df'
    table = f'{x}_Stats'
    table2 = f'{x}_Extra_Stats'
    df = pd.DataFrame(pd.read_sql(f'select * from {table} as a inner join {table2} as b on a.playerplayerid = b.playerid and a.year=b.year', db))
    df_list.append(df)

In [12]:
#Check for-loop output
print(len(df_list))
print(type(df_list[1]))

4
<class 'pandas.core.frame.DataFrame'>


In [13]:
#Create empty dbs to merge from list
qb_df = pd.DataFrame()
rb_df = pd.DataFrame()
te_df = pd.DataFrame()
wr_df = pd.DataFrame()

In [14]:
#Merge dataframes from list
qb_df = pd.concat([df_list[0], qb_df], ignore_index=True)
rb_df = pd.concat([df_list[1], rb_df], ignore_index=True)
te_df = pd.concat([df_list[2], te_df], ignore_index=True)
wr_df = pd.concat([df_list[3], wr_df], ignore_index=True)


In [15]:
#Quick check on QB DF
print(qb_df.head(1))
print(qb_df.columns)

   index  rank  playerplayerid playershortname  playerageexact  \
0      0     1           19801        J. Allen            26.1   

   passattemptspergame  passingyards  completionpercentage  passingtouchdowns  \
0                 37.9          4407                 63.41                 36   

   interceptions  moneythrows  touchdownrate  protectionrate  \
0             15           29           5.58           77.94   

   pressuredcompletionpercentage  airyards  airyardspergame  \
0                          37.83    2785.0           163.82   

   airyardsperattempt  deepballattempts  deepballcompletionpercentage  year  \
0                4.31                80                          42.5  2021   

  position  index  rank  playerid        name team position  played  \
0       QB      0     1     19801  Josh Allen  BUF       QB      17   

   passingcompletions  passingattempts  passingcompletionpercentage  \
0                 409              646                         63.3   

   

In [16]:
#Quick check on RB DF
print(rb_df.head(1))
print(rb_df.columns)

   index  rank  playerplayerid playershortname  playerageexact  carries  \
0      0     1           21682       J. Taylor            23.5      332   

   rushingyards  yardspercarry  rushingtouchdowns  targets  receptions  \
0          1811           5.45                 18       53          40   

   receivingyards  receivingtds  snapshare  opportunityshare  \
0             360             2      68.56             70.77   

   shotguncarryrate  shotgunyardspercarry  undercentercarryrate  \
0             37.65                   1.0                 62.34   

   undercenteryardspercarry  averagedefendersinthebox  stackedfrontcarryrate  \
0                       1.0                      6.07                  13.25   

   basefrontcarryrate  lightfrontcarryrate  year position  index  rank  \
0               28.31                57.83  2021       RB      0     1   

   playerid             name team position  played  rushingattempts  \
0     21682  Jonathan Taylor  IND       RB      17     

In [17]:
#Quick check on WR DF
print(wr_df.head(1))
print(wr_df.columns)

   index  rank  playerplayerid playershortname  targets  receptions  \
0      0     1           18882         C. Kupp      191         145   

   receivingyards  receivingtds  hograte  snapshare  averagetargetdistance  \
0            1947            16      NaN      94.01                   8.32   

   airyards  airyardspergame  airyardsperreception  targets.1  redzonetargets  \
0    1590.0            93.52                  11.0        191              38   

   endzonetargets  targetaccuracy  targetshare  fantasypointspertarget  year  \
0              11            7.76        31.72                     2.3  2021   

  position  index  rank  playerid         name team position  played  \
0       WR      0     1     18882  Cooper Kupp  LAR       WR      17   

   receivingtargets  receptions  receptionpercentage  receivingyards  \
0               191         145                 75.9            1947   

   receivingtouchdowns  receivinglong  receivingyardspertarget  \
0                   

In [18]:
#Quick check on TE DF
print(te_df.head(1))
print(te_df.columns)

   index  rank  playerplayerid playershortname  targets  receptions  \
0      0     1           19803      M. Andrews      154         107   

   receivingyards  receivingtds  hograte  snapshare  averagetargetdistance  \
0            1361             9      NaN      76.54                   9.35   

   airyards  airyardspergame  airyardsperreception  targets.1  redzonetargets  \
0    1440.0             84.7                  13.5        154              20   

   endzonetargets  targetaccuracy  targetshare  fantasypointspertarget  year  \
0              13            7.35        26.64                    1.96  2021   

  position  index  rank  playerid          name team position  played  \
0       TE      0     1     19803  Mark Andrews  BAL       TE      17   

   receivingtargets  receptions  receptionpercentage  receivingyards  \
0               154         107                 69.5            1361   

   receivingtouchdowns  receivinglong  receivingyardspertarget  \
0                 

In [19]:
''' 
Identify duplicate columns 
Code used from this reference:
https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/
'''
def getDuplicateColumns(df):
  
    # Create an empty set
    duplicateColumnNames = set()
      
    # Iterate through all the columns 
    # of dataframe
    for x in range(df.shape[1]):
          
        # Take column at xth index.
        col = df.iloc[:, x]
          
        # Iterate through all the columns in
        # DataFrame from (x + 1)th index to
        # last index
        for y in range(x + 1, df.shape[1]):
              
            # Take column at yth index.
            otherCol = df.iloc[:, y]
               # Check if two columns at x & y
            # index are equal or not,
            # if equal then adding 
            # to the set
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
                  
    # Return list of unique column names 
    # whose contents are duplicates.
    return list(duplicateColumnNames)

In [21]:
# Set dup column array names (organizational purposes)
dupColumnsNames = []
for x in range(0, len(positions)):
    col_name = f'dupColumns_{positions[x]}'
    dupColumnsNames.append(col_name)
dupColumnsNames

['dupColumns_qb', 'dupColumns_rb', 'dupColumns_te', 'dupColumns_wr']

In [22]:
# get columns w/ duplicate values for each dataframe

dupColumns = []
for i in range(0, len(dupColumnsNames)):
    dupColumns.append(getDuplicateColumns(df_list[i]))
dupColumns

[['year', 'passingtouchdowns', 'passinginterceptions', 'position', 'playerid'],
 ['playerid', 'year', 'position', 'receptions'],
 ['year',
  'receptions',
  'targets.1',
  'position',
  'receivingtouchdowns',
  'playerid'],
 ['year', 'receptions', 'targets.1', 'position', 'playerid']]

In [23]:
#Check column names
for frame in df_list:
    print(frame.columns)


Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'passattemptspergame', 'passingyards', 'completionpercentage',
       'passingtouchdowns', 'interceptions', 'moneythrows', 'touchdownrate',
       'protectionrate', 'pressuredcompletionpercentage', 'airyards',
       'airyardspergame', 'airyardsperattempt', 'deepballattempts',
       'deepballcompletionpercentage', 'year', 'position', 'index', 'rank',
       'playerid', 'name', 'team', 'position', 'played', 'passingcompletions',
       'passingattempts', 'passingcompletionpercentage', 'passingyards',
       'passingyardsperattempt', 'passingtouchdowns', 'passinginterceptions',
       'passingrating', 'rushingattempts', 'rushingyards',
       'rushingyardsperattempt', 'rushingtouchdowns', 'fantasypointspergame',
       'fantasypoints', 'year'],
      dtype='object')
Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'carries', 'rushingyards', 'yardspercarry', 'rushing

In [24]:
#Check count of columns per df
i = 0 # mini index
for frame in df_list:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 43
rb df: 45
te df: 46
wr df: 46


In [25]:
# Create new list of df dropping duplicate columns with same name
df_list2 = []
for frame in df_list:
    frame2 = frame.loc[:,~frame.columns.duplicated()].copy()
    df_list2.append(frame2)

In [26]:
#Column count after duplicates dropped
i = 0 # mini index
for frame in df_list2:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 37
rb df: 37
te df: 40
wr df: 40


In [27]:
# Check column names after drops
for frame in df_list2:
    print(frame.columns)

Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'passattemptspergame', 'passingyards', 'completionpercentage',
       'passingtouchdowns', 'interceptions', 'moneythrows', 'touchdownrate',
       'protectionrate', 'pressuredcompletionpercentage', 'airyards',
       'airyardspergame', 'airyardsperattempt', 'deepballattempts',
       'deepballcompletionpercentage', 'year', 'position', 'playerid', 'name',
       'team', 'played', 'passingcompletions', 'passingattempts',
       'passingcompletionpercentage', 'passingyardsperattempt',
       'passinginterceptions', 'passingrating', 'rushingattempts',
       'rushingyards', 'rushingyardsperattempt', 'rushingtouchdowns',
       'fantasypointspergame', 'fantasypoints'],
      dtype='object')
Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'carries', 'rushingyards', 'yardspercarry', 'rushingtouchdowns',
       'targets', 'receptions', 'receivingyards', 'receivingtds', 's

In [28]:
#Check to see if duplicate values still exist- second time
dupColumns2 = []
for i in range(0, len(dupColumnsNames)):
    dupColumns2.append(getDuplicateColumns(df_list2[i]))
dupColumns2

[['passinginterceptions', 'playerid'],
 ['playerid'],
 ['receivingtouchdowns', 'playerid', 'targets.1'],
 ['playerid', 'targets.1']]

In [29]:
#Drop colunmns in dup values list... columns that contain same values as another column w/ different column name
for i in range(0, len(df_list2)):
    df_list2[i].drop(columns = dupColumns2[i], axis = 1, inplace = True)

In [30]:
#Print colun names for check
for frame in df_list2:
    print(frame.columns)

Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'passattemptspergame', 'passingyards', 'completionpercentage',
       'passingtouchdowns', 'interceptions', 'moneythrows', 'touchdownrate',
       'protectionrate', 'pressuredcompletionpercentage', 'airyards',
       'airyardspergame', 'airyardsperattempt', 'deepballattempts',
       'deepballcompletionpercentage', 'year', 'position', 'name', 'team',
       'played', 'passingcompletions', 'passingattempts',
       'passingcompletionpercentage', 'passingyardsperattempt',
       'passingrating', 'rushingattempts', 'rushingyards',
       'rushingyardsperattempt', 'rushingtouchdowns', 'fantasypointspergame',
       'fantasypoints'],
      dtype='object')
Index(['index', 'rank', 'playerplayerid', 'playershortname', 'playerageexact',
       'carries', 'rushingyards', 'yardspercarry', 'rushingtouchdowns',
       'targets', 'receptions', 'receivingyards', 'receivingtds', 'snapshare',
       'opportunityshare'

In [31]:
#Column count after duplicates dropped
i = 0 # mini index
for frame in df_list2:
    print(f'{positions[i]} df: {len(frame.columns)}')
    i += 1

qb df: 35
rb df: 36
te df: 37
wr df: 38


In [32]:
#Drop column named Index. 
for frame in df_list2:
    frame.drop(columns = 'index', axis = 1, inplace= True)

In [33]:
#identify nulls in the data
for frame in df_list2:
    print(frame.isnull().sum())

#QBS - none, WR / TE - Hog Rate, RB - a good amount

rank                             0
playerplayerid                   0
playershortname                  0
playerageexact                   0
passattemptspergame              0
passingyards                     0
completionpercentage             0
passingtouchdowns                0
interceptions                    0
moneythrows                      0
touchdownrate                    0
protectionrate                   0
pressuredcompletionpercentage    0
airyards                         0
airyardspergame                  0
airyardsperattempt               0
deepballattempts                 0
deepballcompletionpercentage     0
year                             0
position                         0
name                             0
team                             0
played                           0
passingcompletions               0
passingattempts                  0
passingcompletionpercentage      0
passingyardsperattempt           0
passingrating                    0
rushingattempts     

In [34]:
#Quick look at the data
for frame in df_list2:
    print(frame.head(1))

   rank  playerplayerid playershortname  playerageexact  passattemptspergame  \
0     1           19801        J. Allen            26.1                 37.9   

   passingyards  completionpercentage  passingtouchdowns  interceptions  \
0          4407                 63.41                 36             15   

   moneythrows  touchdownrate  protectionrate  pressuredcompletionpercentage  \
0           29           5.58           77.94                          37.83   

   airyards  airyardspergame  airyardsperattempt  deepballattempts  \
0    2785.0           163.82                4.31                80   

   deepballcompletionpercentage  year position        name team  played  \
0                          42.5  2021       QB  Josh Allen  BUF      17   

   passingcompletions  passingattempts  passingcompletionpercentage  \
0                 409              646                         63.3   

   passingyardsperattempt  passingrating  rushingattempts  rushingyards  \
0                

In [35]:
#Split positions out of list and into individual dfs
qb_full_stats = df_list2[0]
rb_full_stats = df_list2[1]
te_full_stats = df_list2[2]
wr_full_stats = df_list2[3]

In [37]:
#Drop column w/ Nans for wr and te
wr_full_stats.drop(columns='hograte', axis = 1, inplace=True)
te_full_stats.drop(columns='hograte', axis = 1, inplace=True)

In [38]:
#Check to ensure no more nans
wr_full_stats.isnull().sum()

rank                          0
playerplayerid                0
playershortname               0
targets                       0
receptions                    0
receivingyards                0
receivingtds                  0
snapshare                     0
averagetargetdistance         0
airyards                      0
airyardspergame               0
airyardsperreception          0
redzonetargets                0
endzonetargets                0
targetaccuracy                0
targetshare                   0
fantasypointspertarget        0
year                          0
position                      0
name                          0
team                          0
played                        0
receivingtargets              0
receptionpercentage           0
receivingtouchdowns           0
receivinglong                 0
receivingyardspertarget       0
receivingyardsperreception    0
rushingattempts               0
rushingyards                  0
rushingyardsperattempt        0
rushingt

In [39]:
#Check to ensure no more nans
te_full_stats.isnull().sum()

rank                          0
playerplayerid                0
playershortname               0
targets                       0
receptions                    0
receivingyards                0
receivingtds                  0
snapshare                     0
averagetargetdistance         0
airyards                      0
airyardspergame               0
airyardsperreception          0
redzonetargets                0
endzonetargets                0
targetaccuracy                0
targetshare                   0
fantasypointspertarget        0
year                          0
position                      0
name                          0
team                          0
played                        0
receivingtargets              0
receptionpercentage           0
receivinglong                 0
receivingyardspertarget       0
receivingyardsperreception    0
rushingattempts               0
rushingyards                  0
rushingyardsperattempt        0
rushingtouchdowns             0
fumbles 

In [40]:
#Look at RB stats
rb_full_stats.isnull().sum()

rank                         0
playerplayerid               0
playershortname              0
playerageexact               0
carries                      0
rushingyards                 0
yardspercarry               30
rushingtouchdowns            0
targets                      0
receptions                   0
receivingyards               0
receivingtds                 0
snapshare                    0
opportunityshare             6
shotguncarryrate            58
shotgunyardspercarry        67
undercentercarryrate        58
undercenteryardspercarry    40
averagedefendersinthebox    58
stackedfrontcarryrate       58
basefrontcarryrate          58
lightfrontcarryrate         58
year                         0
position                     0
name                         0
team                         0
played                       0
rushingattempts              0
rushingyardsperattempt       0
receivingtargets             0
receivingtouchdowns          0
fumbles                      0
fumblesl

In [42]:
#Filter to RB that have 0 carries to look at missing values
rb_full_stats[rb_full_stats["carries"] == 0].count()

rank                        52
playerplayerid              52
playershortname             52
playerageexact              52
carries                     52
rushingyards                52
yardspercarry               28
rushingtouchdowns           52
targets                     52
receptions                  52
receivingyards              52
receivingtds                52
snapshare                   52
opportunityshare            52
shotguncarryrate             0
shotgunyardspercarry        28
undercentercarryrate         0
undercenteryardspercarry    28
averagedefendersinthebox     0
stackedfrontcarryrate        0
basefrontcarryrate           0
lightfrontcarryrate          0
year                        52
position                    52
name                        52
team                        52
played                      52
rushingattempts             52
rushingyardsperattempt      52
receivingtargets            52
receivingtouchdowns         52
fumbles                     52
fumblesl

In [44]:
#Fill all nans with 0 since RB had 0 carries
rb_full_stats[rb_full_stats["carries"] == 0] = rb_full_stats[rb_full_stats["carries"] == 0].fillna(value = 0)

In [45]:
#Check to make sure all nulls are filled w/ RBs with 0 carries
rb_full_stats[rb_full_stats["carries"] == 0].isnull().sum()

rank                        0
playerplayerid              0
playershortname             0
playerageexact              0
carries                     0
rushingyards                0
yardspercarry               0
rushingtouchdowns           0
targets                     0
receptions                  0
receivingyards              0
receivingtds                0
snapshare                   0
opportunityshare            0
shotguncarryrate            0
shotgunyardspercarry        0
undercentercarryrate        0
undercenteryardspercarry    0
averagedefendersinthebox    0
stackedfrontcarryrate       0
basefrontcarryrate          0
lightfrontcarryrate         0
year                        0
position                    0
name                        0
team                        0
played                      0
rushingattempts             0
rushingyardsperattempt      0
receivingtargets            0
receivingtouchdowns         0
fumbles                     0
fumbleslost                 0
fantasypoi

In [46]:
qb_full_stats.isnull().sum()

rank                             0
playerplayerid                   0
playershortname                  0
playerageexact                   0
passattemptspergame              0
passingyards                     0
completionpercentage             0
passingtouchdowns                0
interceptions                    0
moneythrows                      0
touchdownrate                    0
protectionrate                   0
pressuredcompletionpercentage    0
airyards                         0
airyardspergame                  0
airyardsperattempt               0
deepballattempts                 0
deepballcompletionpercentage     0
year                             0
position                         0
name                             0
team                             0
played                           0
passingcompletions               0
passingattempts                  0
passingcompletionpercentage      0
passingyardsperattempt           0
passingrating                    0
rushingattempts     

In [47]:
qb_full_stats.head()

,rank,playerplayerid,playershortname,playerageexact,passattemptspergame,passingyards,completionpercentage,passingtouchdowns,interceptions,moneythrows,touchdownrate,protectionrate,pressuredcompletionpercentage,airyards,airyardspergame,airyardsperattempt,deepballattempts,deepballcompletionpercentage,year,position,name,team,played,passingcompletions,passingattempts,passingcompletionpercentage,passingyardsperattempt,passingrating,rushingattempts,rushingyards,rushingyardsperattempt,rushingtouchdowns,fantasypointspergame,fantasypoints
0,1,19801,J. Allen,26.1,37.9,4407,63.41,36,15,29,5.58,77.94,37.83,2785.0,163.82,4.31,80,42.50,2021,QB,Josh Allen,BUF,17,409,646,63.3,6.8,92.17,122,763,6.3,6,23.7,402.58
1,2,21681,J. Herbert,24.3,39.5,5014,65.92,38,15,30,5.65,88.62,41.34,2678.0,157.52,3.98,71,43.66,2021,QB,Justin Herbert,LAC,17,443,672,65.9,7.5,97.66,63,302,4.8,3,22.4,380.76
2,3,4314,T. Brady,44.9,42.3,5316,67.45,43,12,44,5.98,92.98,46.84,2792.0,164.23,3.88,87,40.22,2021,QB,Tom Brady,TB,17,485,719,67.5,7.4,102.08,28,81,2.9,2,22.0,374.74
3,4,18890,P. Mahomes,26.8,38.7,4839,66.26,37,13,32,5.62,85.27,40.65,2219.0,130.52,3.37,79,36.70,2021,QB,Patrick Mahomes,KC,17,436,658,66.3,7.4,98.45,66,381,5.8,2,21.3,361.66
4,6,2593,A. Rodgers,38.6,33.2,4115,68.92,37,4,37,6.96,88.94,39.75,1956.0,122.25,3.68,69,39.13,2021,QB,Aaron Rodgers,GB,16,366,531,68.9,7.7,111.90,33,101,3.1,3,20.8,332.30


In [48]:
rb_full_stats.head()

,rank,playerplayerid,playershortname,playerageexact,carries,rushingyards,yardspercarry,rushingtouchdowns,targets,receptions,receivingyards,receivingtds,snapshare,opportunityshare,shotguncarryrate,shotgunyardspercarry,undercentercarryrate,undercenteryardspercarry,averagedefendersinthebox,stackedfrontcarryrate,basefrontcarryrate,lightfrontcarryrate,year,position,name,team,played,rushingattempts,rushingyardsperattempt,receivingtargets,receivingtouchdowns,fumbles,fumbleslost,fantasypointspergame,fantasypoints
0,1,21682,J. Taylor,23.5,332,1811,5.45,18,53,40,360,2,68.56,70.77,37.65,1.0,62.34,1.0,6.07,13.25,28.31,57.83,2021,RB,Jonathan Taylor,IND,17,332,5.5,51,2,4,2,19.6,333.1
1,12,19562,A. Ekeler,27.1,206,911,4.42,12,94,70,647,8,67.10,67.41,34.46,1.0,65.53,1.0,6.65,19.90,37.37,41.74,2021,RB,Austin Ekeler,LAC,16,206,4.4,94,8,4,3,17.1,273.8
2,3,18858,J. Mixon,26.0,292,1205,4.12,13,48,42,314,3,67.57,74.88,25.34,1.0,74.65,1.0,6.69,17.80,34.93,47.26,2021,RB,Joe Mixon,CIN,16,292,4.1,48,3,2,1,15.4,245.9
3,4,21768,N. Harris,24.3,307,1200,3.90,7,94,74,467,3,84.45,86.42,52.44,1.0,46.57,1.0,6.52,16.93,29.64,50.81,2021,RB,Najee Harris,PIT,17,307,3.9,94,3,0,0,13.3,226.7
4,24,18983,J. Conner,27.2,202,752,3.72,15,39,37,375,3,57.93,58.92,84.65,1.0,15.34,1.0,6.29,12.87,21.78,64.35,2021,RB,James Conner,ARI,15,202,3.7,39,3,2,0,14.7,220.7


In [49]:
te_full_stats.head()

,rank,playerplayerid,playershortname,targets,receptions,receivingyards,receivingtds,snapshare,averagetargetdistance,airyards,airyardspergame,airyardsperreception,redzonetargets,endzonetargets,targetaccuracy,targetshare,fantasypointspertarget,year,position,name,team,played,receivingtargets,receptionpercentage,receivinglong,receivingyardspertarget,receivingyardsperreception,rushingattempts,rushingyards,rushingyardsperattempt,rushingtouchdowns,fumbles,fumbleslost,fantasypointspergame,fantasypoints
0,1,19803,M. Andrews,154,107,1361,9,76.54,9.35,1440.0,84.70,13.5,20,13,7.35,26.64,1.96,2021,TE,Mark Andrews,BAL,17,154,69.5,43,8.8,12.7,1,0,0.0,0,1,0,11.4,194.1
1,2,15048,T. Kelce,135,92,1125,9,83.09,7.19,971.0,60.68,10.6,16,4,7.74,22.31,1.95,2021,TE,Travis Kelce,KC,16,134,68.7,69,8.4,12.2,2,3,1.5,1,1,1,10.7,170.8
2,3,19920,D. Schultz,104,78,808,8,82.59,7.29,759.0,44.64,9.7,14,7,8.17,16.35,2.01,2021,TE,Dalton Schultz,DAL,17,104,75.0,32,7.8,10.4,0,0,0.0,0,0,0,7.7,130.8
3,4,19063,G. Kittle,95,71,910,6,92.35,6.70,637.0,45.50,9.0,8,5,7.68,24.86,2.08,2021,TE,George Kittle,SF,14,94,75.5,48,9.7,12.8,3,20,6.7,0,2,1,9.1,127.0
4,7,10974,R. Gronkowski,89,55,802,6,76.19,10.53,938.0,78.16,17.1,12,8,7.69,17.21,1.92,2021,TE,Rob Gronkowski,TB,12,89,61.8,42,9.0,14.6,0,0,0.0,0,0,0,9.7,116.2


In [50]:
wr_full_stats.head()

,rank,playerplayerid,playershortname,targets,receptions,receivingyards,receivingtds,snapshare,averagetargetdistance,airyards,airyardspergame,airyardsperreception,redzonetargets,endzonetargets,targetaccuracy,targetshare,fantasypointspertarget,year,position,name,team,played,receivingtargets,receptionpercentage,receivingtouchdowns,receivinglong,receivingyardspertarget,receivingyardsperreception,rushingattempts,rushingyards,rushingyardsperattempt,rushingtouchdowns,fumbles,fumbleslost,fantasypointspergame,fantasypoints
0,1,18882,C. Kupp,191,145,1947,16,94.01,8.32,1590.0,93.52,11.0,38,11,7.76,31.72,2.30,2021,WR,Cooper Kupp,LAR,17,191,75.9,16,59,10.2,13.4,4,18,4.5,0,0,0,17.3,294.50
1,3,20932,D. Samuel,120,77,1405,6,81.44,8.09,971.0,60.68,12.6,10,6,7.14,27.77,2.82,2021,WR,Deebo Samuel,SF,16,121,63.6,6,83,11.6,18.2,59,365,6.2,8,4,2,16.4,261.96
2,5,22564,J. Chase,128,81,1455,13,86.11,11.96,1532.0,90.11,18.9,12,12,7.30,23.70,2.38,2021,WR,Ja'Marr Chase,CIN,17,128,63.3,13,82,11.4,18.0,7,21,3.0,0,2,1,13.2,223.60
3,4,21685,J. Jefferson,167,108,1616,10,89.01,11.95,1996.0,117.41,18.5,23,15,7.41,29.87,1.98,2021,WR,Justin Jefferson,MIN,17,167,64.7,10,56,9.7,15.0,6,14,2.3,0,1,1,13.1,222.40
4,2,16470,D. Adams,169,123,1553,11,80.33,8.99,1520.0,95.00,12.4,28,11,7.33,31.58,2.04,2021,WR,Davante Adams,GB,16,169,72.8,11,59,9.2,12.6,0,0,0.0,0,0,0,13.8,221.30


In [61]:
#Check more null columns for RBs
rb_full_stats.isnull().sum()

rank                         0
playerplayerid               0
playershortname              0
playerageexact               0
carries                      0
rushingyards                 0
yardspercarry                0
rushingtouchdowns            0
targets                      0
receptions                   0
receivingyards               0
receivingtds                 0
snapshare                    0
opportunityshare             6
shotguncarryrate             6
shotgunyardspercarry         0
undercentercarryrate         6
undercenteryardspercarry    16
averagedefendersinthebox     6
stackedfrontcarryrate        6
basefrontcarryrate           6
lightfrontcarryrate          6
year                         0
position                     0
name                         0
team                         0
played                       0
rushingattempts              0
rushingyardsperattempt       0
receivingtargets             0
receivingtouchdowns          0
fumbles                      0
fumblesl

In [60]:
rb_full_stats["yardspercarry"].fillna('rushingyardsperattempt', inplace = True)

In [62]:
rb_full_stats.fillna('0', inplace = True)


In [63]:
#Hand picked duplicate values. Columns aren't identical, but close enough
wr_dups = ["targets", "receivingtds"]
rb_dups = ["carries", "rushingyardsperattempt", "receivingtargets", "receivingtds"]
te_dups = ["targets"]
qb_dups = ["passingcompletionpercentage"]

In [64]:
# # Drop redundant columns from previous line
qb_full_stats.drop(columns = qb_dups, axis= 1, inplace= True)
rb_full_stats.drop(columns = rb_dups, axis= 1, inplace= True)
te_full_stats.drop(columns = te_dups, axis= 1, inplace= True)
wr_full_stats.drop(columns = wr_dups, axis= 1, inplace= True)

In [65]:
#Uploading full stats to AWS database
qb_full_stats.to_sql("qb_full_stats", con, if_exists="replace")
wr_full_stats.to_sql("wr_full_stats", con, if_exists="replace")
rb_full_stats.to_sql("rb_full_stats", con, if_exists="replace")
te_full_stats.to_sql("te_full_stats", con, if_exists="replace")
